In [ ]:
%pip install pymongo
%pip install python-dotenv

Importación de librerias

In [2]:
import pymongo
import requests
import time

import os
from datetime import datetime
from dotenv import load_dotenv

Carga los secretos del archivo **.env**

In [ ]:
# Carga los secretos del archivo .env
load_dotenv()

# Accede a los valores definidos en .env
APIKEY_ETHERSCAN = os.getenv("APIKEY_ETHERSCAN")
MONGODB_SERVER = os.getenv("MONGODB_SERVER")

print(APIKEY_ETHERSCAN[:3] + '*****' + APIKEY_ETHERSCAN[-3:])
print(MONGODB_SERVER[:10] + '*****' + MONGODB_SERVER[20:])

### Conexión MongoDB

In [4]:
mongoClient = pymongo.MongoClient(MONGODB_SERVER, serverSelectionTimeoutMS=5000)

try:
    print(mongoClient.server_info())
except Exception:
    print("Unable to connect to the server.")

{'version': '7.0.5', 'gitVersion': '7809d71e84e314b497f282ea8aa06d7ded3eb205', 'modules': [], 'allocator': 'tcmalloc', 'javascriptEngine': 'mozjs', 'sysInfo': 'deprecated', 'versionArray': [7, 0, 5, 0], 'openssl': {'running': 'OpenSSL 3.0.2 15 Mar 2022', 'compiled': 'OpenSSL 3.0.2 15 Mar 2022'}, 'buildEnvironment': {'distmod': 'ubuntu2204', 'distarch': 'x86_64', 'cc': '/opt/mongodbtoolchain/v4/bin/gcc: gcc (GCC) 11.3.0', 'ccflags': '-Werror -include mongo/platform/basic.h -ffp-contract=off -fasynchronous-unwind-tables -g2 -Wall -Wsign-compare -Wno-unknown-pragmas -Winvalid-pch -gdwarf-5 -fno-omit-frame-pointer -fno-strict-aliasing -O2 -march=sandybridge -mtune=generic -mprefer-vector-width=128 -Wno-unused-local-typedefs -Wno-unused-function -Wno-deprecated-declarations -Wno-unused-const-variable -Wno-unused-but-set-variable -Wno-missing-braces -fstack-protector-strong -gdwarf64 -Wa,--nocompress-debug-sections -fno-builtin-memcmp -Wimplicit-fallthrough=5', 'cxx': '/opt/mongodbtoolchain/

Definición BBDD, colecciones e indices en MongoDB

In [5]:
mongoDb = mongoClient['web3_tfm']
collection_tx = mongoDb['account_transactions']
collection_methods = mongoDb['methods']
collection_block_history = mongoDb['block_history']

# Crear índice único en 'hash'
collection_tx.create_index('hash', unique=True)

# Crear índices en 'date' y 'to'
collection_tx.create_index('date')
collection_tx.create_index('to')

# Crear índice único en 'methodId'
collection_methods.create_index('methodId', unique=True)

# Crear índice único para 'date' e indice para 'max_block_number'
collection_block_history.create_index('date')
collection_block_history.create_index('max_block_number')

'max_block_number_1'

Valida si ya existe el hash (clave única) en la colección

In [6]:
def hash_exists(collection, hash_value):
    return collection.find_one({"hash": hash_value}) is not None

Valida si ya existe el método (clave única) en la colección

In [7]:
def method_exists(collection, method_value):
    return collection.find_one({"methodId": method_value}) is not None

Obtiene transacciones con llamada a API

In [8]:
def get_transactions(url):
    
    headers = {'Content-Type': 'application/json'}
    response = requests.get(url, headers=headers)

    # Imprime el estado de la respuesta y el contenido
    print(response.status_code)
    
    return response

Convierte timestamp en fecha

In [9]:
def timestamp_to_date(timestamp):
    timestamp_int = int(timestamp)
    fecha_hora = datetime.fromtimestamp(timestamp_int)
    fecha_str = fecha_hora.strftime("%Y-%m-%d %H:%M:%S")
    return fecha_str

# Prueba
prueba = timestamp_to_date(1705866815)
print(type(prueba), prueba)

<class 'str'> 2024-01-21 20:53:35


In [10]:
def from_wei_to_ether(wei_value):
    ether_value = int(wei_value) / 10**18
    return ether_value

In [11]:
def from_wei_to_gwei(wei_value):
    gwei_value = int(wei_value) / 10**9
    return gwei_value

Redes blockchain (https://chainlist.org/):
- 1 (0x1) - Ethereum
- 42161 (0xa4b1) - Arbitrum One
- 42170 (0xa4ba) - Arbitrum Nova
- 10 (0xa) - Optimism Mainnet
- 137 (0x89) - Polygon Mainnet

In [12]:
def insert_transactions(transactions, from_network, to_network):

    # transactions = data['result']
    methods = {}
    lower_block = 99999999
    lower_date = "9999-12-31"

    for transaction in transactions:
        
        tx = {
            'hash': transaction['hash'],
            'date': timestamp_to_date(transaction["timeStamp"]),
            'from': transaction['from'],
            'to': transaction['to'],
            'from_network': from_network,
            'to_network': to_network,
            'value': from_wei_to_ether(transaction['value']),
            'blockNumber': int(transaction['blockNumber']),
            'nonce': int(transaction['nonce']),
            'transactionIndex': int(transaction['transactionIndex']),
            'gas': int(transaction['gas']),
            'gasUsed': int(transaction['gasUsed']),
            'gasPrice': from_wei_to_gwei(transaction['gasPrice']),
            'cumulativeGasUsed': int(transaction['cumulativeGasUsed']),
            'isError': transaction['isError'],
            'txreceipt_status': transaction['txreceipt_status'],
            # 'input': transaction['input'],
            'contractAddress': transaction['contractAddress'],
            'methodId': transaction['methodId']
        }
        
        if tx["blockNumber"] < lower_block:
            lower_block = tx["blockNumber"]
            lower_date = tx["date"]
        
        if ("methodId" in transaction) and ("functionName" in transaction):
            methods[transaction["methodId"]] = transaction["functionName"]
            # del transaction["functionName"]
        
        # Inserta en collection_tx
        if hash_exists(collection_tx, tx["hash"]):
            print("AVISO. Hash ya existe: ", tx["hash"], " Bloque: ", tx["blockNumber"])
        else:
            collection_tx.insert_one(tx)
        
    print(methods)

    # Guardar methods en tablas
    for methodId, functionName in methods.items():
        element = {"methodId": methodId, "functionName": functionName}
        
        # Inserta en collection_methods
        if not method_exists(collection_methods, methodId):
            collection_methods.insert_one(element)    
            
    return lower_block, lower_date      

In [13]:
def save_transactions(url, from_network, to_network):
    
    print("save_transactions: ", url)
    
    response = get_transactions(url)

    # Imprime el estado de la respuesta y el contenido
    print("status_code: ", response.status_code)
    
    if response.status_code == 200:
        data = response.json()
        transactions = data['result']
        lower_block, lower_date = insert_transactions(transactions, from_network, to_network)
    else:
        lower_block = 99999999
        lower_date = "0001-01-01"
        
    return lower_block, lower_date

In [14]:
def generate_url_api(address, start_block=0, end_block=99999999, page=1):

    url = f"https://api.etherscan.io/api?module=account&action=txlist&address={address}" \
                f"&startblock={start_block}&endblock={end_block}&page={page}&offset=1000" \
                f"&sort=desc&apikey={APIKEY_ETHERSCAN}"
    return url

In [15]:
def process_transactions(address, start_block, end_block, page, from_network, to_network):
    
    url = generate_url_api(address, start_block, end_block, page)
    
    lower_block, lower_date = save_transactions(url, from_network, to_network)
    print("page: ", page, "| lower_block: ", lower_block, " | lower_date:", lower_date)
    
    return lower_block, lower_date

In [16]:
def process_account_transactions(address, start_block, end_block, end_date="1900-01-01", from_network=0, to_network=0):
    
    lower_date = "9999-12-31"
    
    while (end_date <= lower_date):

        for page in range(1, 6):

            lower_block, lower_date = process_transactions(address, start_block, end_block, page, from_network, to_network)
            
            if lower_date == "0001-01-01" or lower_block == 99999999:
                break

        print("Pausa 1 segundo")
        time.sleep(1)  # Pausa de 1 segundo

        for page in range(6, 11):
            
            lower_block, lower_date = process_transactions(address, start_block, end_block, page, from_network, to_network)
            
            if lower_date == "0001-01-01" or lower_block == 99999999:
                break
            
        if lower_block == 99999999:
            break
        else:
            end_block = lower_block - 1

In [ ]:
address = "0x99c9fc46f92e8a1c0dec1b1747d010903e884be1" # Optimism Gateway | L1StandardBridgeProxy
start_block = 0
end_block =  99999999
end_date = "2022-12-31"
from_network = 1 # Ethereum
to_network = 10 # Optimism

process_account_transactions(address, start_block, end_block, end_date, from_network, to_network)

In [ ]:
address = "0x4dbd4fc535ac27206064b68ffcf827b0a60bab3f" # Arbitrum: Delayed Inbox
start_block = 0
end_block =  99999999
end_date = "2022-12-31"
from_network = 1 # Ethereum
to_network = 42161 # Arbitrum

process_account_transactions(address, start_block, end_block, end_date, from_network, to_network)

In [ ]:
address = "0x3154Cf16ccdb4C6d922629664174b904d80F2C35" # Base: Base Bridge | L1StandardBridge
start_block = 0
end_block =  99999999
end_date = "2022-12-31"
from_network = 1 # Ethereum
to_network = 8453 # Base

process_account_transactions(address, start_block, end_block, end_date, from_network, to_network)

In [ ]:
address = "0xd19d4B5d358258f05D7B411E21A1460D11B0876F" # Linea: L1 Message Service 
start_block = 0
end_block =  18018024  # 99999999
end_date = "2022-12-31"
from_network = 1 # Ethereum
to_network = 59144 # Linea

process_account_transactions(address, start_block, end_block, end_date, from_network, to_network)

Función que devuelve el intervalo de timestamps para una fecha

In [ ]:
from datetime import datetime, timedelta

def dates_from_timestamp(date_str):

    fecha = datetime.strptime(date_str, "%Y-%m-%d")

    # Obtener el timestamp Unix correspondiente a la medianoche de ese día (el inicio del día)
    inicio_del_dia = fecha.replace(hour=0, minute=0, second=0, microsecond=0)
    timestamp_inicio = int(inicio_del_dia.timestamp())

    # Obtener el timestamp Unix correspondiente a la medianoche del día siguiente (el final del día)
    final_del_dia = inicio_del_dia + timedelta(days=1)
    timestamp_final = int(final_del_dia.timestamp())

    return timestamp_inicio, timestamp_final

print(dates_from_timestamp("2024-01-21"))